In [15]:
from dotenv import load_dotenv
import os

# Load the .env file only if it exists
dotenv_path = '/workspace/general_projects/database_generator/.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)
    print(f"Loaded environment variables from {dotenv_path}")
else:
    print(f"No .env file found at {dotenv_path}, relying on system environment variables.")

# Access the environment variable, with a fallback
path_to_logs = os.getenv('PATH_TO_SAVE_THE_LOGS')
print(f"Logs will be saved to: {path_to_logs}")

# Access the environment variable, with a fallback
path_to_logs = os.getenv('PATH_TO_SAVE_THE_OUTCOMES')
print(f"Logs will be saved to: {path_to_logs}")

Loaded environment variables from /workspace/general_projects/database_generator/.env
Logs will be saved to: /workspace/general_projects/database_generator/notebooks/tmp
Logs will be saved to: /workspace/general_projects/database_generator/notebooks/tmp


In [16]:
import os
print(os.getenv("PATH_TO_SAVE_THE_LOGS"))

/workspace/general_projects/database_generator/notebooks/tmp


In [17]:
print(os.environ)

environ({'HOSTNAME': 'c38c0673b4b7', 'HOME': '/root', 'PYTHONUNBUFFERED': '1', 'GPG_KEY': 'A035C8C19219BA821ECEA86B64E628F8D684696D', 'PYTHON_SHA256': '07a4356e912900e61a15cb0949a06c4a05012e213ecd6b4e84d0f67aabbee372', 'PATH': '/workspace/general_projects/database_generator/.venv/bin:/vscode/vscode-server/bin/linux-x64/e8653663e8840adaf45af01eab5c627a5af81807/bin/remote-cli:/root/.pyenv/shims:/root/.pyenv/bin:/root/.local/bin:/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', 'LANG': 'C.UTF-8', 'SHELL': '/bin/bash', 'PYTHON_VERSION': '3.11.10', 'PWD': '/vscode/vscode-server/bin/linux-x64/e8653663e8840adaf45af01eab5c627a5af81807', 'PYENV_ROOT': '/root/.pyenv', 'VSCODE_CWD': '/vscode/vscode-server/bin/linux-x64/e8653663e8840adaf45af01eab5c627a5af81807', 'VSCODE_NLS_CONFIG': '{"userLocale":"en","osLocale":"en","resolvedLanguage":"en","defaultMessagesFile":"/vscode/vscode-server/bin/linux-x64/e8653663e8840adaf45af01eab5c627a5af81807/out/nls.messages.json","locale

In [18]:
from database_generator.data_generator import IndustrialPumpData
from datetime import datetime, timezone

start_datetime = datetime(2025, 1, 1, 0, 0, tzinfo=timezone.utc)
end_datetime = datetime(2025, 1, 4, 0, 0, tzinfo=timezone.utc)
frequency = '1h'
seed_for_random = 42
pump_data = IndustrialPumpData(
    start_datetime=start_datetime,
    end_datetime=end_datetime,
    frequency=frequency,
    seed_for_random=seed_for_random
)


In [19]:
pump_data

IndustrialPumpData(start_datetime=datetime.datetime(2025, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), end_datetime=datetime.datetime(2025, 1, 4, 0, 0, tzinfo=datetime.timezone.utc), frequency='1h', seed_for_random=42, include_flag=True)

In [20]:
standard_pump_data = pump_data.generate_standard_data()

[database_generator.data_generator] 2025-01-23 05:24:35 - INFO: A new standard dataset was created


In [21]:
standard_pump_data.columns

Index(['temperature_c', 'pressure_mpa', 'vibration_mm_s', 'flow_rate_l_min',
       'humidity_%', 'flag_normal_data'],
      dtype='object')

In [22]:
from database_generator.data_generator import ExponentialAnomaly

anomaly_in_pressure = ExponentialAnomaly(
    start_datetime= datetime(2025, 1, 1, 0, 0, tzinfo=timezone.utc),
    end_datetime= datetime(2025, 1, 1, 23, 0, tzinfo=timezone.utc),
    variable_to_insert_anomalies='pressure_mpa',
    standard_data=standard_pump_data
)

In [23]:
pressure_with_anomaly = anomaly_in_pressure.introduce_anomaly()

[database_generator.data_generator] 2025-01-23 05:24:36 - DEBUG: Number of rows affected by the anomaly: 24
[database_generator.data_generator] 2025-01-23 05:24:36 - INFO:  A new dataset with 24 anomalies was created


In [24]:
pressure_with_anomaly.tail()

,temperature_c,pressure_mpa,vibration_mm_s,flow_rate_l_min,humidity_%,flag_normal_data
timestamp,,,,,,
2025-01-01 19:00:00+00:00,73.587696,3.198301,2.967710,302.273701,45.493884,False
2025-01-01 20:00:00+00:00,76.465649,3.201030,3.048970,304.420697,44.127082,False
2025-01-01 21:00:00+00:00,74.774224,3.227646,2.999454,309.657441,44.067548,False
2025-01-01 22:00:00+00:00,75.067528,3.184202,2.954695,299.511570,46.527394,False
2025-01-01 23:00:00+00:00,73.575252,3.300303,3.019278,295.940784,40.105019,False


In [25]:
pressure_with_anomaly.head()

,temperature_c,pressure_mpa,vibration_mm_s,flow_rate_l_min,humidity_%,flag_normal_data
timestamp,,,,,,
2025-01-01 00:00:00+00:00,75.496714,3.073265,3.005294,293.082596,37.053176,False
2025-01-01 01:00:00+00:00,74.861736,2.880885,3.024148,294.693763,44.248010,False
2025-01-01 02:00:00+00:00,75.647689,3.055707,2.935258,302.263525,41.785077,False
2025-01-01 03:00:00+00:00,76.523030,3.020922,3.036813,301.593703,36.535452,False
2025-01-01 04:00:00+00:00,74.765847,3.030014,2.919924,301.378553,44.497999,False


In [26]:
# Create an IntermittentSpikeAnomaly instance
from database_generator.data_generator import IntermittentSpikeAnomaly

spike_anomaly = IntermittentSpikeAnomaly(
    start_datetime=datetime(2025, 1, 2, 0, 0, tzinfo=timezone.utc),
    end_datetime=datetime(2025, 1, 2, 23, 59, tzinfo=timezone.utc),
    variable_to_insert_anomalies="pressure_mpa",
    standard_data=standard_pump_data,
    spike_fraction=0.30,
    spike_multiplier=100.0,
    seed_for_random=42
)

# Introduce anomalies
spike_data = spike_anomaly.introduce_anomaly()

[database_generator.data_generator] 2025-01-23 05:24:37 - DEBUG: Number of rows in the specified time range: 24
[database_generator.data_generator] 2025-01-23 05:24:37 - INFO: Introduced 7 spike anomalies in 'pressure_mpa'.


In [27]:
spike_data

,temperature_c,pressure_mpa,vibration_mm_s,flow_rate_l_min,humidity_%,flag_normal_data
timestamp,,,,,,
2025-01-02 08:00:00+00:00,74.986503,7.564503,3.048376,298.473341,34.969913,False
2025-01-02 16:00:00+00:00,75.738467,7.564503,3.102224,301.208414,40.379023,False
2025-01-02 00:00:00+00:00,74.455617,7.564503,3.029197,295.367462,43.409765,False
2025-01-02 18:00:00+00:00,74.884352,7.564503,2.983007,295.603693,44.875599,False
2025-01-02 11:00:00+00:00,73.779156,7.564503,2.913684,295.463353,43.958313,False
2025-01-02 09:00:00+00:00,73.942289,7.564503,3.221294,312.551982,33.929057,False
2025-01-02 13:00:00+00:00,73.040330,7.564503,3.040563,303.175781,43.141728,False


In [28]:
from database_generator.data_generator import SimpleDatabaseFactory
list_of_dfs = [standard_pump_data,pressure_with_anomaly, spike_data]
factory = SimpleDatabaseFactory(list_of_df=list_of_dfs, flag_column='flag_normal_data')
final_database = factory.create_database()

In [29]:
final_database.head()

,temperature_c,pressure_mpa,vibration_mm_s,flow_rate_l_min,humidity_%,flag_normal_data
timestamp,,,,,,
2025-01-02 13:00:00+00:00,73.040330,7.564503,3.040563,303.175781,43.141728,False
2025-01-01 00:00:00+00:00,75.496714,3.073265,3.005294,293.082596,37.053176,False
2025-01-01 01:00:00+00:00,74.861736,2.880885,3.024148,294.693763,44.248010,False
2025-01-01 02:00:00+00:00,75.647689,3.055707,2.935258,302.263525,41.785077,False
2025-01-01 03:00:00+00:00,76.523030,3.020922,3.036813,301.593703,36.535452,False


In [30]:
final_database.tail()

,temperature_c,pressure_mpa,vibration_mm_s,flow_rate_l_min,humidity_%,flag_normal_data
timestamp,,,,,,
2025-01-02 06:00:00+00:00,74.398293,2.965903,3.005170,300.405977,35.908897,True
2025-01-02 05:00:00+00:00,74.708306,2.985781,3.102419,304.278104,42.975785,True
2025-01-02 04:00:00+00:00,74.399361,2.984974,2.967451,301.533713,40.484980,True
2025-01-02 03:00:00+00:00,75.375698,2.925474,2.950833,302.451015,39.349285,True
2025-01-03 03:00:00+00:00,74.614918,3.033194,3.090810,300.436689,37.465284,True
